In [14]:
import math
import pandas as pd

# Climate Modeling

Notes about climate modeling. [MIT](https://computationalthinking.mit.edu/Fall20/lecture20/)'s open courses have been a huge resource in this endeavor.

# [Introduction to Climate Modeling](https://youtu.be/Gi4ZZVS2GLA)

## Zero Dimensional Energy Balance Model

### 1) Background Climate Physics

Simplest climate model: 

$\text{change in heat content} = \text{absorbed solar radiation} - \text{outgoing thermal radiation} + \text{human caused greenhouse effect}$

By interpreting as an average over entire globe, you can create a "zero dimensional" model.

### 1.1) Absorbed Solar Radiation

The power of the sun's rays that intercept the earth, $S$, is equal to $\frac{W}{m^2}$ (energy p/ unit time p/ unit area)

A small fraction, $\alpha$, of solar radiation is reflected back to space. Remaining fraction ($1-\alpha$) is absorbed.

Since incoming solar rays $\approx$ parallel, the cross-sectional area of earth that intercepts them is a disc, of area $\pi R^2$. All other terms act on spherical surface area, $4\pi R^2$, so absorbed solar radiation p/ unit surface area is reduced by a factor of 4. Vid explains this a little, but I'm taking it for granted

Thus, $\text{absorbed solar radiation} \equiv \frac{S(1-\alpha)}{4}$

In [15]:
S = 1369
alpha = 0.3
def absorbed_solar_radiation(S=S, alpha=alpha):
    return S * (1-alpha) / 4

### 1.2) Outgoing Thermal Radiation

The outgoing thermal radiation term, $G(T)$ (or "blackbody cooling to space") represents the combined effects of *negative feedbacks that dampen warming*, such as **blackbody radiation**, and *positive feedbacks that amplify warming*, such as the **water vapor feedback**.

Physics are complicated, so we *linearize* the model by considering only the 1st term of a Taylor Series expansion

$G(T) \approx G(T_0) + G'(T_0)(T-T_0) = G'(T_0)T + G(T_0) - G'(T+0)(T_0)$

$T_0$, the pre-industrial equilibrium temperature $\approx 14 \degree C$

Simplify the expression by defining

$A \equiv G(T_0) - G'(T_0)(T_0)$
$B \equiv G'(T_0)$ 

Where A is a constant, and B is the climate feedback parameter, giving

$\text{outgoing thermal radiation} \equiv G(T) \approx A - BT$

In [16]:
T0 = 14
def outgoing_thermal_radiation(T, A, B):
    return A - B * T

Climate Feedback Parameter $B \frac{W}{m^2 \degree C}$

Since $B<0$, overall climate feedback is negative (stabilizing). Positive feedbacks increase $B$, and reduce the efficiency with which Earth cools by radiating thermal energy to space, thus amplifying warming.

In [17]:
B = -1.3

$A$, is the model's equilibrium, in this case, the preindustrial equilibrium.

$\text{absorbed solar radiation} = \text{outgoing thermal radiation}$

or

$\frac{S(1-\alpha)}{4} = A - BT_0$

In [18]:
A = S * (1. - alpha) / 4 + B * T0

### 1.3) Human-caused greenhouse effect

Greenhouse effect is a logarithmic function of gaseous $CO_2$ concentrations

* $a$: $CO_2$ forcing coefficient $(\frac{W}{m^2})$
* $[CO_2]_\pi$: preindustrial $CO_2$ concentration $(\text{ppm})$

$\text{greenhouse effect} = a \cdot ln(\frac{[CO_2]}{[CO_2]_\pi})$

In [19]:
def greenhouse_effect(CO2, a, CO2_PI):
    return a * math.log(CO2/CO2_PI)

In [20]:
a = 5.0
CO2_PI = 280.0

### 1.4) Change in Heat Content

The heat content $CT$ is determined by the temperature $T$ (in Kelvin) and the heat capacity of the climate system. While we are interested in the temperature of the atmosphere, which has a very small heat capacity, its heat is closely coupled with that of the upper ocean, which has a much larger heat capacity of $\approx 51 \frac{J}{m^2 \degree C}$

In [21]:
C = 51.0

The *change in heat content over time* is thus simply given by $\frac{d(CT)}{dt}$. Since the heat capacity of sea water hardly changes with temperature, we can rewrite this in terms of the change in temperature with time as:

${\text{change in heat content } =\; C \frac{dT}{dt}}$

### 1.5) 0-Dimensional Climate Model Equation

Ordinary Differential Equation:

$\begin{gather}
{C \frac{dT}{dt}}
\; = \; \frac{(1 - α)S}{4}
\; - \; (A - BT)
\; + \; a \ln \left( \frac{[\text{CO}₂]}{[\text{CO}₂]_{\text{PI}}} \right),
\end{gather}$

## 2) Numerical solution method and data structures

### 2.1) Discretization

The energy balance model equation above can be **discretized** in time as

$C \frac{T(t+Δt) - T(t)}{\Delta t} = \frac{\left( 1-\alpha \right) S}{4} - (A - BT(t)) + a \ln \left( \frac{[\text{CO}₂](t)}{[\text{CO}₂]_{\text{PI}}} \right).$

Our **finite difference equation**, which results from a first-order truncation of the Taylor series expansion, approximates the exact **ordinary differential equation** above in the limit that $\Delta t \rightarrow 0$. In practice, we can keep decreasing $\Delta t$ until the solution converges within a tolerable error.
Hereafter, we use the subscript $n$ to denote the $n$-th timestep, where $T_{n+1} \equiv T(t_{n+1})$ denotes the temperature at the next timestep $t_{n+1} = t_{n} + \Delta t$.

By re-arranging the equation, we can solve for the temperature at the next timestep $n+1$ based on the known temperature at the present timestep $n$:

$T_{n+1} = T_{n} + \frac{\Delta t}{C} \left[ \frac{ \left( 1-\alpha \right) S}{4} - (A - BT_{n}) + a \ln \left( \frac{[\text{CO}₂]_{n}}{[\text{CO}₂]_{\text{PI}}} \right) \right]$

### 2.2) Timestepping
More generally, we recognize this equation to be of the form:
$T_{n+1} = T_{n} + \Delta t * \text{tendency}(T_{n} \,; ...),$
which we implement below (don't forget to update the time as well, $t_{n+1} = t_{n} + \Delta t$), which takes in an instance of our anticipated energy balance model `EBM` type as its only argument.

In [22]:
# dataframe w/ the model value, time, and the rate of time change for each step
ebm = pd.DataFrame(
                { 'T': [T0] },
                { 'time': [0] },
                { 'delta_t': 0.01 }
                )

def time_step(ebm):
    # calculate T_n+1
    ebm['T'].append(ebm['T'][-1] + ebm['delta_t'] * tendency(ebm))
    # t += delta_t
    ebm['time'].append(ebm['time'][-1] + ebm['delta_t'])

def tendency(ebm):
    (ebm['delta_t'] / C) * (
        + absorbed_solar_radiation(S, alpha)
        - outgoing_thermal_radiation(ebm['T'][-1], A, B)
        + greenhouse_effect( CO2(emb['T'][-1]), a, CO2_PI )
    )